In [1]:
import gmql as gl

In [2]:
# loading of a dataset
data_path = "/home/luca/Documenti/resources/hg_narrowPeaks"
output_path = "/home/luca/Documenti/resources/result_project"

parser = gl.parsers.NarrowPeakParser()

data = gl.GMQLDataset(parser=parser)\
                .load_from_path(path=data_path, meta_load=False)
    
# selects all the data coming from cell K562 and having antibody H3K4me3
condition = (data.MetaField("cell") == 'K562') & (data.MetaField("antibody") == 'H3K4me3')
filtered_data = data.meta_select(predicate=condition) # test

In [3]:
# project on metadata
proj_data = filtered_data.meta_project(attr_list=['antibody', 'cell'])

In [4]:
# project on region fields
new_reg_field = {
    "new_attribute" : (proj_data.RegField("pValue") + proj_data.RegField("signalValue"))/2
}

In [5]:
proj_data = proj_data.reg_project(field_list=['score', 'signalValue', 'pValue'], new_field_dict=new_reg_field)

In [6]:
# materialization
output_path = "/home/luca/Documenti/resources/result_project"
proj_data = proj_data.materialize(output_path=output_path)

2017-04-23 15:55:28,299 - PyGML logger - INFO - Loading meta data from path /home/luca/Documenti/resources/result_project/exp/


100%|██████████| 12/12 [00:00<00:00, 1256.06it/s]


2017-04-23 15:55:28,513 - PyGML logger - INFO - dataframe construction


100%|██████████| 3/3 [00:00<00:00, 24.11it/s]

2017-04-23 15:55:28,682 - PyGML logger - INFO - Loading region data from path /home/luca/Documenti/resources/result_project/exp/



100%|██████████| 14/14 [00:12<00:00,  1.17it/s]


In [7]:
proj_data.regs

,chr,new_attribute,pValue,score,signalValue,start,stop,strand
id_sample,,,,,,,,
-1499507608156314203,chr1,74.458500,63.91700,0.0,85.0,137700,137850,*
-1499507608156314203,chr1,91.988250,89.97650,0.0,94.0,138420,138570,*
-1499507608156314203,chr1,77.988250,89.97650,0.0,66.0,138680,138830,*
-1499507608156314203,chr1,87.494500,101.98900,0.0,73.0,138940,139090,*
-1499507608156314203,chr1,85.494500,101.98900,0.0,69.0,139360,139510,*
-1499507608156314203,chr1,7.714870,6.42974,0.0,9.0,235580,235730,*
-1499507608156314203,chr1,9.992190,9.98438,0.0,10.0,540620,540770,*
-1499507608156314203,chr1,8.259820,6.51964,0.0,10.0,569840,569990,*
-1499507608156314203,chr1,46.888700,71.77740,0.0,22.0,713000,713150,*
